In [1]:
import cv2
from keras.models import load_model
import numpy as np
import os

Using TensorFlow backend.


In [2]:
model = load_model('emojinator.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [3]:
def main():
    emojis = get_emojis()
    cap = cv2.VideoCapture(0)
    x, y, w, h = 300, 50, 350, 350

    while (cap.isOpened()):
        ret, img = cap.read()
        img = cv2.flip(img, 1)
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        mask2 = cv2.inRange(hsv, np.array([2, 50, 60]), np.array([25, 150, 255]))
        res = cv2.bitwise_and(img, img, mask=mask2)
        gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
        median = cv2.GaussianBlur(gray, (5, 5), 0)

        kernel_square = np.ones((5, 5), np.uint8)
        dilation = cv2.dilate(median, kernel_square, iterations=2)
        opening = cv2.morphologyEx(dilation, cv2.MORPH_CLOSE, kernel_square)
        ret, thresh = cv2.threshold(opening, 30, 255, cv2.THRESH_BINARY)

        thresh = thresh[y:y + h, x:x + w]
        contours,_ = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
        if len(contours) > 0:
            contour = max(contours, key=cv2.contourArea)
            if cv2.contourArea(contour) > 2500:
                x, y, w1, h1 = cv2.boundingRect(contour)
                newImage = thresh[y:y + h1, x:x + w1]
                newImage = cv2.resize(newImage, (50, 50))
                pred_probab, pred_class = keras_predict(model, newImage)
                print(pred_class, pred_probab)
                img = overlay(img, emojis[pred_class], 400, 250, 90, 90)

        x, y, w, h = 300, 50, 350, 350
        cv2.imshow("Frame", img)
        cv2.imshow("Contours", thresh)
        k = cv2.waitKey(10)
        if k == 27:
            break

In [4]:

def keras_predict(model, image):
    processed = keras_process_image(image)
    pred_probab = model.predict(processed)[0]
    pred_class = list(pred_probab).index(max(pred_probab))
    return max(pred_probab), pred_class

In [5]:
def keras_process_image(img):
    image_x = 50
    image_y = 50
    img = cv2.resize(img, (image_x, image_y))
    img = np.array(img, dtype=np.float32)
    img = np.reshape(img, (-1, image_x, image_y, 1))
    return img

In [6]:
def get_emojis():
    emojis_folder = 'hand_emo/'
    emojis = []
    for emoji in range(len(os.listdir(emojis_folder))):
        print(emoji)
        emojis.append(cv2.imread(emojis_folder+str(emoji)+'.png', -1))
    return emojis


In [7]:
def overlay(image, emoji, x,y,w,h):
    emoji = cv2.resize(emoji, (w, h))
    try:
        image[y:y+h, x:x+w] = blend_transparent(image[y:y+h, x:x+w], emoji)
    except:
        pass
    return image

In [8]:
def blend_transparent(face_img, overlay_t_img):
    # Split out the transparency mask from the colour info
    overlay_img = overlay_t_img[:,:,:3] # Grab the BRG planes
    overlay_mask = overlay_t_img[:,:,3:]  # And the alpha plane

    # Again calculate the inverse mask
    background_mask = 255 - overlay_mask

    # Turn the masks into three channel, so we can use them as weights
    overlay_mask = cv2.cvtColor(overlay_mask, cv2.COLOR_GRAY2BGR)
    background_mask = cv2.cvtColor(background_mask, cv2.COLOR_GRAY2BGR)

    # Create a masked out face image, and masked out overlay
    # We convert the images to floating point in range 0.0 - 1.0
    face_part = (face_img * (1 / 255.0)) * (background_mask * (1 / 255.0))
    overlay_part = (overlay_img * (1 / 255.0)) * (overlay_mask * (1 / 255.0))

    # And finally just add them together, and rescale it back to an 8bit integer image
    return np.uint8(cv2.addWeighted(face_part, 255.0, overlay_part, 255.0, 0.0))


In [ ]:
keras_predict(model, np.zeros((50, 50, 1), dtype=np.uint8))
main()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
11 0.9956801
11 0.9954527
11 0.9936911
11 0.99210876
11 0.99545324
11 0.9875187
11 0.9875187
11 0.978618
11 0.98257786
11 0.98154366
11 0.98011094
11 0.98208195
11 0.98011094
11 0.98011094
11 0.9852588
11 0.9875187
11 0.9778456
11 0.982578
11 0.9836068
11 0.97960746
11 0.9653045
11 0.98257726
11 0.98043287
11 0.98011094
11 0.98011094
11 0.9732168
11 0.9733609
11 0.9784514
11 0.970301
11 0.978618
11 0.97193074
11 0.97038805
11 0.97861505
11 0.9653409
11 0.981989
11 0.985485
11 0.9875187
11 0.97314745
11 0.9801113
11 0.9784388
11 0.986186
11 0.9792707
11 0.97772455
11 0.9756497
11 0.9778451
11 0.986267
11 0.9653045
11 0.9720329
11 0.9778456
11 0.98011094
11 0.9795234
11 0.9718241
11 0.98011094
11 0.97707796
11 0.9770781
11 0.9790147
11 0.97454655
11 0.98011094
11 0.9753829
11 0.9770781
11 0.97538936
11 0.973644
11 0.9795926
11 0.9770783
11 0.986267
11 0.9753847
11 0.9796831
11 0.97586524
11 0.97959614
11 0.9764287
11 0.9776718
11 0.97538924
11 0.9758916

11 0.9695482
11 0.87864715
11 0.99809486
11 0.9990169
11 0.98612183
11 0.66737694
11 0.9626026
11 0.9999478
11 0.999456
11 0.9985821
11 0.99958295
11 0.9995394
11 0.79561424
11 0.9992036
11 0.99824727
11 0.9710072
11 0.99762493
11 0.99979776
11 0.97295105
11 0.9995171
11 0.966891
11 0.75960255
11 0.975879
11 0.9941248
11 0.99028695
11 0.999647
11 0.9998235
11 0.9986719
11 0.99791414
11 0.999913
11 0.9984218
11 0.9964361
11 0.99971586
11 0.99761826
11 0.99998546
11 0.9999491
11 0.9998186
11 0.9997696
11 0.99990475
11 0.99770594
11 0.9962528
11 0.9999106
11 0.9994711
11 0.9963574
11 0.9799012
11 0.9996971
11 0.9999913
11 0.99850947
11 0.99999857
11 0.99992573
11 0.9992593
11 0.9999144
11 0.98131
11 0.98520535
11 0.9999194
11 0.9996681
11 0.9998348
11 0.999882
11 0.9999999
11 0.99999964
11 0.9996691
11 0.9998147
11 0.9987606
11 0.99975246
11 0.9983524
11 0.99995255
11 0.9995272
11 0.9999546
11 0.9999474
11 0.99973863
11 0.9971929
11 0.9959045
11 0.99955076
11 0.99998
11 0.9999876
11 0.999

11 0.58920985
11 0.6660129
11 0.5880856
11 0.66602
11 0.7512695
11 0.609902
11 0.54532343
11 0.5406735
11 0.6462475
11 0.5897289
11 0.66579264
11 0.589209
11 0.66601235
11 0.6236853
11 0.5653006
11 0.5860176
11 0.73991245
11 0.6238968
11 0.7398932
11 0.7907359
11 0.7657271
11 0.78862715
11 0.8029316
11 0.70987535
11 0.78913414
11 0.7772766
11 0.77791864
11 0.77748185
11 0.77928853
11 0.77748185
11 0.795914
11 0.8983737
11 0.77748185
11 0.77728885
11 0.8615547
11 0.9004384
11 0.82386416
11 0.88010186
11 0.78739285
11 0.873318
11 0.9133653
11 0.92640007
11 0.8803101
11 0.8743885
11 0.9604807
11 0.96006894
11 0.94474816
11 0.9021284
11 0.95846003
11 0.9367866
11 0.8904673
11 0.89514184
11 0.93738836
11 0.93697417
11 0.9390394
11 0.97744375
11 0.96820134
11 0.96820134
11 0.9606072
11 0.9682415
11 0.9643168
11 0.97524333
11 0.9786211
11 0.95383036
11 0.9654961
11 0.9606083
11 0.9517371
11 0.951737
11 0.9517372
11 0.97878486
11 0.9517371
11 0.87329024
11 0.91378236
11 0.866483
11 0.8752602
1

11 0.96550727
11 0.94862753
11 0.9417828
11 0.9628458
11 0.9689269
11 0.9463798
11 0.9181173
11 0.65414995
11 0.696165
11 0.6483047
11 0.6488523
11 0.5887834
14 0.6287951
14 0.9459663
14 0.9050532
11 0.5657387
14 0.5279579
11 0.7350824
14 0.89814454
11 0.9992778
11 0.9999548
11 0.99460346
14 0.9964641
14 0.99992526
14 0.9854473
11 0.9674888
11 0.9941083
11 0.94954413
11 0.7538684
11 0.76180726
14 0.5023183
14 0.6288756
14 0.837744
14 0.82204294
14 0.939342
14 0.96182317
14 0.9840656
14 0.9864
14 0.9959305
14 0.9963659
14 0.99791545
14 0.99748755
14 0.9975267
14 0.99108446
14 0.9918035
14 0.9899128
14 0.99032825
14 0.9822711
14 0.97321993
14 0.963828
14 0.9532976
14 0.9102075
14 0.87378585
14 0.9096503
14 0.91640335
14 0.9277107
14 0.9326789
14 0.9325241
14 0.9229503
14 0.9168467
14 0.8543274
14 0.8601898
14 0.8543293
14 0.7724535
11 0.55807763
11 0.6006462
14 0.6271793
14 0.75981814
14 0.7667259
14 0.7375897
14 0.74454534
14 0.7259819
14 0.72211295
11 0.62520814
11 0.54117745
11 0.7573

14 0.70810807
14 0.8419407
14 0.60640687
14 0.60640687
14 0.5713542
14 0.60640687
14 0.5446032
14 0.60640687
11 0.5215519
11 0.53050387
11 0.5361207
11 0.5305038
14 0.53803325
14 0.5013109
14 0.6054808
14 0.6059976
14 0.57137597
14 0.53803325
14 0.57137597
14 0.571376
14 0.6064069
14 0.57137585
14 0.75224215
14 0.60640687
14 0.60640687
14 0.5977486
14 0.60640484
14 0.60640687
14 0.57137597
14 0.6196146
14 0.60640687
11 0.53049403
14 0.60640687
14 0.53392917
11 0.5029766
11 0.53050387
11 0.53050387
14 0.5700525
14 0.60640687
14 0.57149976
